# KNN model for predicting racket age using datasets P1, P2, and P3, based on **energy per band features** - Sound

### Model Description

This notebook implements a K-Nearest Neighbors (KNN) model to predict the age of a racket (P1, P2, P3) based on sound features extracted from audio files. The workflow involves reading `.wav` files, **extracting energy per frequency band** using FFT, and training the model using these features. The model's performance is evaluated using accuracy metrics and visualized through scatter plots and a confusion matrix.

### Import libraries

In [4]:
import os
import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.io import wavfile
from scipy.fft import fft
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks
import librosa
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
# import openpyxl

### Import Tools Functions

In [5]:
import sys
# Reach the project root
notebook_path = os.path.abspath('')
project_root = os.path.abspath(os.path.join(notebook_path, '../../../'))
functions_path = os.path.join(project_root, 'Functions')

# Add Functions folder
if functions_path not in sys.path:
    sys.path.append(functions_path)

Energy

In [6]:
from energy_per_frequency_band_from_spectrum import energy_per_frequency_band_from_spectrum

# Main

In [7]:
import ast

# Read the CSV file
df_combined = pd.read_csv("../../../Data/Vibration/All_Data_combinedv3.csv")


df_combined.head()

,Raw Signal Ch0,Spectrum,freqs,File Name,Position,Racket Type,Age
0,[-0.01573961 -0.02470204 -0.0184555 ... -6.36...,"[5415.435319490299, 3187.673624625387, 2567.48...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_1.csv,C,RB,1
1,[ 0.10185837 0.10090781 0.08882211 ... -6.35...,"[3574.0039456266313, 2253.172677296551, 2189.4...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_2.csv,C,RB,1
2,[-0.02701054 -0.01506063 -0.02510942 ... -6.09...,"[4336.317505980091, 2636.183071505374, 2297.29...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_3.csv,C,RB,1
3,[-0.05687327 -0.06352719 -0.05049093 ... -9.29...,"[7218.047965980523, 4371.60331681831, 3234.473...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_S_1.csv,S,RB,1
4,[-0.08674804 -0.04994775 -0.05334261 ... -8.81...,"[6351.72555524914, 3431.4594161592727, 3006.71...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_S_2.csv,S,RB,1


In [27]:
# Determine the minimum number of samples for each age group
min_samples_per_age = df_combined['Age'].value_counts().min()

# Create a new DataFrame with equal samples for each age group
df_sorted_homogeneous = (
    df_combined.groupby('Age')
    .apply(lambda x: x.sample(n=min_samples_per_age, random_state=42))
    .reset_index(drop=True)
)

df_sorted_homogeneous

# Verify the distribution
print(df_sorted_homogeneous['Age'].value_counts())

sample_rate=20000
spectrum=np.array(df_sorted_homogeneous["Spectrum"].to_list())
spectrum = np.array([np.array(eval(element)) for element in spectrum])

freqs = np.fft.fftfreq(len(spectrum), d=1/sample_rate)[:len(spectrum) // 2]  # Compute frequency bins


band_energies, band_frequencies = energy_per_frequency_band_from_spectrum(spectrum, freqs, 20)
print(band_energies)
X = band_energies 




# Encode string labels into integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_sorted_homogeneous["Age"])

# # print(y)
# print(len(X),len(y))

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X,y)


Age
1    216
2    216
3    216
Name: count, dtype: int64
[764476393.9010029, 2785443703.908447, 0.0, 160673947.37589058, 1379204520.5828822, 0.0, 1786470056.958582, 807964321.9233088, 0.0, 4660368399.359713, 2090165090.2463565, 0.0, 3537264509.4676657, 122841072.49706066, 0.0, 2143789506.85185, 830709492.8966856, 0.0, 4751591167.388882, 0.0, 3221752722.106924, 1394194004.4153335, 0.0, 609323318.8951746, 2157883403.297183, 0.0, 3247566060.2260566, 2583882865.3883276, 0.0, 1014813754.0833795, 2589027586.0584073, 0.0, 239239037.43042392, 1406199014.1466115, 0.0, 1738721540.9315593, 0.0, 1127258132.3492384, 2500995270.8037167, 0.0, 1745334817.2472262, 1752862625.9333355, 0.0, 1805425433.263696, 4261279916.209197, 0.0, 2990741274.543765, 1447725572.6713552, 0.0, 856122107.7117486, 711916389.4521891, 0.0, 1941473176.2607915, 0.0, 2352527665.335547, 872428669.4468434, 0.0, 537925796.3281739, 2490913829.585513, 0.0, 1147966550.7088704, 1500817755.7529914, 0.0, 660551550.4012238, 1637727539.911

ValueError: Found input variables with inconsistent numbers of samples: [499, 648]

In [5]:
# Définir les bonnes valeurs trouvé
n_neighbors = 10  # Nombre de voisins
weights_options = ['uniform']  # Méthode de pondération
metric_options = ['manhattan']  # Métriques


# Définir les plages de valeurs pour les hyperparamètres
n_neighbors_range = range(1, 21)  # Nombre de voisins
weights_options = ['uniform', 'distance']  # Méthode de pondération
metric_options = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']  # Métriques

# Liste pour stocker les résultats
results = []


# Tester toutes les combinaisons d'hyperparamètres
for n_neighbors in n_neighbors_range:
    for weights in weights_options:
        for metric in metric_options:
            # Créer et entraîner le modèle KNN
            knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
            knn.fit(X_train, y_train)

            # Évaluer sur l'ensemble de test
            y_pred = knn.predict(X_test)
            accuracy_test = accuracy_score(y_test, y_pred)

            # Évaluer sur l'ensemble d'entraînement
            y_train_pred = knn.predict(X_train)
            accuracy_train = accuracy_score(y_train, y_train_pred)

            # Ajouter les résultats à la liste
            results.append({
                'n_neighbors': n_neighbors,
                'weights': weights,
                'metric': metric,
                'accuracy_train': accuracy_train,
                'accuracy_test': accuracy_test
            })

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)



print(results_df)

     n_neighbors   weights     metric  accuracy_train  accuracy_test
0              1   uniform  euclidean        1.000000       0.784615
1              1   uniform  manhattan        1.000000       0.784615
2              1   uniform  chebyshev        1.000000       0.738462
3              1   uniform  minkowski        1.000000       0.784615
4              1  distance  euclidean        1.000000       0.784615
..           ...       ...        ...             ...            ...
155           20   uniform  minkowski        0.754826       0.738462
156           20  distance  euclidean        1.000000       0.761538
157           20  distance  manhattan        1.000000       0.746154
158           20  distance  chebyshev        1.000000       0.700000
159           20  distance  minkowski        1.000000       0.761538

[160 rows x 5 columns]
